## Aliexpress

登录
https://login.aliexpress.com/buyer.htm?spm=2114.11010108.1000002.7.650c649b4nvOsb&return=https%3A%2F%2Fwww.aliexpress.com%2F&from=lighthouse&random=E788229D7AAA5AB86F562D4C2561B6C1

分类
https://www.aliexpress.com/category/100003086/jeans.html?spm=2114.search0103.0.0.5cb043cfwWsKOU&site=glo&SortType=total_tranpro_desc&g=y&tag=

In [39]:
from selenium import webdriver
import time,random,re,csv
from pyquery import PyQuery as pq
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

def get_detail(driver,username,password,save_file,login_url,page_url):

    driver.get(page_url) # 访问网页
    print('open crawl page... {}'.format(page_url))
    products = driver.find_elements_by_css_selector('.info')
    
    with open(save_file,'a',encoding='utf-8') as w:
        i = 0
        for p in products:
            i += 1
            item_db = p.find_element_by_css_selector('.product').text
            item_name = re.sub(r',','',item_db)
            print('getting No.{} item, name:{}'.format(i,item_name))

            try:
                reviews = driver.find_element_by_css_selector('.rate-num').text
                review_db = re.search('\d+',reviews).group(0)
                review_num = re.sub(r',','',review_db)
                print('getting item reviews:{}'.format(review_db))            
            except NoSuchElementException :
                print('no item reviews')
                review_num = 0
                review_db = 0

            try:
                price_info = driver.find_element_by_css_selector('.value').text
                price = re.sub(r',|US','',price_info).strip()
                print('getting item price:{}'.format(price))  
            except NoSuchElementException:
                print('no price')
                price = 'no price'
                price_db = ''

            try:
                orders = driver.find_element_by_css_selector('.order-num-a').text
                order_db = re.search('\d+',orders).group(0)
                order_num = re.sub(r',','',order_db)
                print('getting item orders number:{}'.format(order_db))    
            except NoSuchElementException :
                print('no item orders')
                order_db = 0
                order_num = 0
                
            item_url = p.find_element_by_css_selector('.product').get_attribute('href')
            print('getting item url: {} '.format(item_url))

            product = {
                'item_name': item_db,
                'price': price,
                'order_num':order_db,
                'review_num': review_db,
                'item_url': item_url,
            }
            #save_to_mongo(product)
            w.write('{},{},{},{},{}\n'.format(item_name,price,order_num,review_num,item_url))

            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'aliexpress'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')
                   
            
            

def aliexpress(username,password,save_file,login_url,page_format,page_num):
    
    driver = webdriver.Chrome()  #声明浏览器对象
    driver.get(login_url) # 访问网页
    print('open login page... {}'.format(login_url))
    driver.switch_to.frame(driver.find_element_by_id('alibaba-login-box'))
    time.sleep(2)
    print('switch to login window... ')
    driver.find_element_by_id('fm-login-id').send_keys(username) # 提取账号输入框，并输入用户名
    print('enter username... ')
    time.sleep(random.randint(2,4))
    driver.find_element_by_id('fm-login-password').send_keys(password) # 提取密码输入框，并输入密码
    print('enter password... ')
    driver.find_element_by_id('fm-login-submit').submit() # 点击登陆按钮 
    print('click login button... ')
    time.sleep(2)
        
    with open(save_file,'a',encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Item name','Price','Order num','Review num','Item url'])
    
    for p in range(page_num):
        current_page = p+1
        print('Getting No.{} page'.format(current_page))
        page_url = page_format.format(current_page)
        get_detail(driver,username,password,save_file,login_url,page_url)
    
    driver.close()
    print('getting item details done')

    
username = '' # 你的账号
password = '' # 你的密码
login_url = 'https://login.aliexpress.com/buyer.htm?spm=2114.11010108.1000002.7.650c649b4nvOsb&return=https%3A%2F%2Fwww.aliexpress.com%2F&from=lighthouse&random=E788229D7AAA5AB86F562D4C2561B6C1'
page_format= 'https://www.aliexpress.com/category/100003086/jeans/{}.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag='
save_file = 'aliexpress_result.csv'
page_num = 2
    
aliexpress(username,password,save_file,login_url,page_format,page_num)

open login page... https://login.aliexpress.com/buyer.htm?spm=2114.11010108.1000002.7.650c649b4nvOsb&return=https%3A%2F%2Fwww.aliexpress.com%2F&from=lighthouse&random=E788229D7AAA5AB86F562D4C2561B6C1
switch to login window... 
enter username... 
enter password... 
click login button... 
Getting No.1 page
open crawl page... https://www.aliexpress.com/category/100003086/jeans/1.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag=
Getting No.2 page
open crawl page... https://www.aliexpress.com/category/100003086/jeans/2.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag=
getting No.1 item, name:Asstseries 2018 Autumn Retro Hole Jeans Men Ankle-Length Pants Cotton Denim Trouser
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/2018-Autumn-New-Retro-Hole-Jeans-Men-Ankle-Length-Pants-Cotton-Denim-Trouser-Male-Plus-Size/32901839714.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_1

getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/Men-Jeans-Pants-Pantacourt-Homme-Mens-Classic-Black-Calca-Masculina-Ofertas-Calientes-Con-Envio-Gratis-Winter/32890987872.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103,searchweb201603_35,ppcSwitch_0&algo_expid=362aa388-8eeb-4eb3-a7d9-06968506df7b-13&algo_pvid=362aa388-8eeb-4eb3-a7d9-06968506df7b 
getting No.15 item, name:KSTUN Jeans Business Casual Male Denim Pants Straight Slim Fit Dark Blue Trousers
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/KSTUN-New-Arrivals-Jeans-Men-Quality-Brand-Business-Casual-Male-Denim-Pants-Straight-Slim-Fit-Dark/32834083942.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_10890_319_10546_317_10548_106

getting No.28 item, name:Newsosoo Dropshipping Distressed Stretch Ripped Biker Jeans Men Hip Hop Slim Fit
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/High-Street-Biker-Jeans-Men-s-Distressed-Stretch-Ripped-Biker-Jeans-Men-Hip-Hop-Slim-Fit/32824446229.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103,searchweb201603_35,ppcSwitch_0&algo_expid=362aa388-8eeb-4eb3-a7d9-06968506df7b-27&algo_pvid=362aa388-8eeb-4eb3-a7d9-06968506df7b 
getting No.29 item, name:Brother Wang 2019 Spring Summer Men Casual Stretch Skinny Jeans Slim fit Pants
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/Men-s-jeans-2015-new-fashion-solid-color-stretch-skinny-jeans-Feet-pants-Male-casual-trousers/32439604185.html?ws_ab_

getting item url: https://www.aliexpress.com/item/2018-New-Fashion-Boutique-Stretch-Casual-Mens-Jeans-Skinny-Jeans-Men-Straight-Mens-Denim-Jeans-Male/32920846092.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103,searchweb201603_35,ppcSwitch_0&algo_expid=362aa388-8eeb-4eb3-a7d9-06968506df7b-40&algo_pvid=362aa388-8eeb-4eb3-a7d9-06968506df7b 
getting No.42 item, name:wrsenqi Full Length Spring Styles Men Jeans On Hot Sales Discount Male Pants
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/Full-Length-Spring-Styles-Men-Jeans-On-Hot-Sales-Discount-Male-Pants/32855166532.html?ws_ab_test=searchweb0_0,searchweb201602_1_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103,searchweb201603_35,ppcSwitch_0&algo_e

## pyquery

In [46]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.common.exceptions import TimeoutException


def get_detail(driver,username,password,save_file,login_url,page_url):
    time.sleep(random.randint(2,4))
    driver.get(page_url) # 访问网页
    print('open crawl page... {}'.format(page_url))
    time.sleep(2)
    html = driver.page_source
    doc = pq(html)
    products = doc('.info')
    print('this page has {} items'.format(len(products)))
    
    with open(save_file,'a',encoding='utf-8') as w:
        i = 0
        for p in products.items():
            i += 1
            item_db = p.find('.product').text()
            item_name = re.sub(r',','',item_db)
            print('getting No.{} item, name:{}'.format(i,item_name))
            
            reviews = p.find('.rate-num').text()
            if reviews == '':
                print('no item reviews')
                review_num = 0
                review_db = 0                
            else:
                review_db = re.search('\d+',reviews).group(0)
                review_num = re.sub(r',','',review_db)
                print('getting item reviews:{}'.format(review_db))            

            price_info = p.find('.value').text()
            if price_info == '':
                print('no price')
                price = 'no price'
                price_db = ''
            else:
                price = re.sub(r',|US','',price_info).strip()
                print('getting item price:{}'.format(price))                  

            orders = p.find('.order-num-a').text()
            if orders == '':
                print('no item orders')
                order_db = 0
                order_num = 0
            else:                
                order_db = re.search('\d+',orders).group(0)
                order_num = re.sub(r',','',order_db)
                print('getting item orders number:{}'.format(order_db))    

            item_url = 'https:'+p.find('.product').attr('href')
            item_url = re.sub(r',','',item_url)
            
            print('getting item url: {} '.format(item_url))
            
            product = {
                'item_name': item_db,
                'price': price,
                'order_num':order_db,
                'review_num': review_db,
                'item_url': item_url,
            }
            save_to_mongo(product)
            w.write('{},{},{},{},{}\n'.format(item_name,price,order_num,review_num,item_url))

            
def save_to_mongo(result):
    
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'aliexpress'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')
        

def aliexpress(username,password,save_file,login_url,page_format,page_num):
    
    driver = webdriver.Chrome()  #声明浏览器对象
    driver.get(login_url) # 访问网页
    print('open login page... {}'.format(login_url))
    driver.switch_to.frame(driver.find_element_by_id('alibaba-login-box'))
    time.sleep(random.randint(2,4))
    print('switch to login window... ')
    driver.find_element_by_id('fm-login-id').send_keys(username) # 提取账号输入框，并输入用户名
    print('enter username... ')
    time.sleep(random.randint(2,4))
    driver.find_element_by_id('fm-login-password').send_keys(password) # 提取密码输入框，并输入密码
    print('enter password... ')
    driver.find_element_by_id('fm-login-submit').submit() # 点击登陆按钮 
    print('click login button... ')
    time.sleep(random.randint(4,6))
    with open(save_file,'a',encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Item name','Price','Order num','Review num','Item url'])
    for p in range(page_num):
        current_page = p+1
        print('Getting No.{} page'.format(current_page))
        page_url = page_format.format(current_page)
        get_detail(driver,username,password,save_file,login_url,page_url)
        time.sleep(10)
    driver.close()
    print('getting item details done')

    
username = '' # 你的账号
password = '' # 你的密码
login_url = 'https://login.aliexpress.com/buyer.htm?spm=2114.11010108.1000002.7.650c649b4nvOsb&return=https%3A%2F%2Fwww.aliexpress.com%2F&from=lighthouse&random=E788229D7AAA5AB86F562D4C2561B6C1'
page_format= 'https://www.aliexpress.com/category/100003086/jeans/{}.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag='
save_file = 'aliexpress_jean_20page_item_details.csv'
page_num = 20

aliexpress(username,password,save_file,login_url,page_format,page_num)

open login page... https://login.aliexpress.com/buyer.htm?spm=2114.11010108.1000002.7.650c649b4nvOsb&return=https%3A%2F%2Fwww.aliexpress.com%2F&from=lighthouse&random=E788229D7AAA5AB86F562D4C2561B6C1
switch to login window... 
enter username... 
enter password... 
click login button... 
Getting No.1 page
open crawl page... https://www.aliexpress.com/category/100003086/jeans/1.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag=
this page has 48 items
getting No.1 item, name:Jack Jones JackJones 2019 Winter Elastic Cotton Stretch Jeans Pants Loose Fit Denim
getting item reviews:3090
getting item price:$27.00
getting item orders number:4365
getting item url: https://www.aliexpress.com/item/JackJones-2019-Winter-New-Men-s-Elastic-Cotton-Stretch-Jeans-Pants-Loose-Fit-Denim-Trousers-Men/32880169150.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb

save to mongodb
getting No.19 item, name:SELECTED Men's Cotton-blend Slight Stretch Whiskers Wash Effect Slim Fit Jeans C
getting item reviews:23
getting item price:$37.42
getting item orders number:667
getting item url: https://www.aliexpress.com/item/SELECTED-Men-s-Cotton-blend-Slight-Stretch-Whiskers-Wash-Effect-Slim-Fit-Jeans-C-418132525/32887784723.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=7029a4c3-996e-4512-89de-22bafb56daff-18&algo_pvid=7029a4c3-996e-4512-89de-22bafb56daff 
save to mongodb
getting No.20 item, name:mrpick Men Stretch Destroyed Ripped Design Ankle Zipper Skinny Jeans For Men E5020
getting item reviews:47
getting item price:$18.80
getting item orders number:602
getting item url: https://www.aliexpress.com/item/Men-Jeans-Stretch-Destroyed-Ripped-Design-Fashion-Ankle-Zipper-Skinny-Jeans-For-Men-

save to mongodb
getting No.32 item, name:Asstseries Designer Skinny Jeans Men Straight Dark Blue Khaki Color Jeans Pant
getting item reviews:158
getting item price:$9.13
getting item orders number:411
getting item url: https://www.aliexpress.com/item/Fashion-Designer-Skinny-Jeans-Men-Straight-Dark-Blue-Khaki-Color-Printed-Mens-Casual-Biker-Denim-Jeans/1000005935884.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=7029a4c3-996e-4512-89de-22bafb56daff-31&algo_pvid=7029a4c3-996e-4512-89de-22bafb56daff 
save to mongodb
getting No.33 item, name:Brother Wang 2019 Winter Men Warm Slim Elasticity Skinny Black Jeans Casual Thick
getting item reviews:223
getting item price:$24.00
getting item orders number:409
getting item url: https://www.aliexpress.com/item/2019-Winter-New-Men-Warm-Slim-Jeans-Elasticity-Skinny-Black-Jeans-Fashio

Getting No.2 page
open crawl page... https://www.aliexpress.com/category/100003086/jeans/2.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag=
this page has 48 items
getting No.1 item, name:Asstseries 2018 Autumn Retro Hole Jeans Men Ankle-Length Pants Cotton Denim Trouser
getting item reviews:222
getting item price:$17.00
getting item orders number:341
getting item url: https://www.aliexpress.com/item/2018-Autumn-New-Retro-Hole-Jeans-Men-Ankle-Length-Pants-Cotton-Denim-Trouser-Male-Plus-Size/32901839714.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=03bf9188-363d-44c5-9e3d-96624958811a-0&algo_pvid=03bf9188-363d-44c5-9e3d-96624958811a 
save to mongodb
getting No.2 item, name:DANTBULUN 2018 New Ripped Jeans Patchwork Hollow Out Printed Beggar Cropped Man
getting item reviews:117
getting item price:$9.15 - 12.

save to mongodb
getting No.23 item, name:NIBESSER 2018 Harem Men Casual Washed Shinny Denim Jeans Male Ripped Pencil Pants
getting item reviews:73
getting item price:$12.24 - 14.86
getting item orders number:256
getting item url: https://www.aliexpress.com/item/NIBESSER-2018-Harem-Jeans-Men-Casual-Washed-Shinny-Denim-Jeans-Male-Fashion-Ripped-Hip-Hop-Sportswear/32855012963.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=03bf9188-363d-44c5-9e3d-96624958811a-22&algo_pvid=03bf9188-363d-44c5-9e3d-96624958811a 
save to mongodb
getting No.24 item, name:Balplein Classical Army High Street Cotton Jogger Designer Big Pocket Military Jeans
getting item reviews:95
getting item price:$29.60
getting item orders number:254
getting item url: https://www.aliexpress.com/item/Fashion-Classical-Army-Pants-High-Street-Cotton-Jeans-Men-Jogg

...

save to mongodb
getting No.18 item, name:YOFEAI Streetwear Vintage Blue Skinny Destroyed Ripped Jeans 2018 Broken Punk Pants
no item reviews
getting item price:$16.48
getting item orders number:22
getting item url: https://www.aliexpress.com/item/Streetwear-Men-s-Jeans-Vintage-Blue-Skinny-Destroyed-Ripped-Jeans-2018-Broken-Punk-Pants-Homme-Hip/32950253772.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=fac5a5f1-1c63-4ed5-9ef8-e2d5b637fd34-17&algo_pvid=fac5a5f1-1c63-4ed5-9ef8-e2d5b637fd34 
save to mongodb
getting No.19 item, name:Arrow Car 2018 Autumn Slim Straight Smoke Casual Black Gray Denim Pants Washed Jeans
getting item reviews:8
getting item price:$14.88
getting item orders number:22
getting item url: https://www.aliexpress.com/item/2018-Autumn-Slim-Straight-Smoke-Gray-Men-s-Jeans-Fashion-Casual-Men-s-Black-And-Gr

getting item reviews:4
getting item price:$18.87
getting item orders number:22
getting item url: https://www.aliexpress.com/item/New-Men-s-Ripped-Side-Ankle-Zipper-Skinny-Stretch-Fashion-Jeans/32868768798.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=fac5a5f1-1c63-4ed5-9ef8-e2d5b637fd34-34&algo_pvid=fac5a5f1-1c63-4ed5-9ef8-e2d5b637fd34 
save to mongodb
getting No.36 item, name:Newsosoo 2018 Straight Ripped Jeans For Men Zipper Fly Denim Designer Pants Jean
getting item reviews:5
getting item price:$23.48
getting item orders number:22
getting item url: https://www.aliexpress.com/item/2018-Fashion-Brand-Mens-Jeans-Straight-Ripped-Jeans-For-Men-Zipper-Fly-Denim-Jeans-Men-Fashion/32857683167.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_53

Getting No.20 page
open crawl page... https://www.aliexpress.com/category/100003086/jeans/20.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&tag=
this page has 48 items
getting No.1 item, name:YFMDH Jeans for Men Spring Hole Ripped Jeans Slim Thin Skinny 2018 Long Pencil Pants
getting item reviews:4
getting item price:$7.20 - 8.28
getting item orders number:21
getting item url: https://www.aliexpress.com/item/Men-s-Fashion-Jeans-for-Men-Spring-Hole-Ripped-Jeans-Slim-Thin-Skinny-2018-Long-Pencil/32914391446.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=4d313ff1-32bd-4bfa-93ee-f4594f55b838-0&algo_pvid=4d313ff1-32bd-4bfa-93ee-f4594f55b838 
save to mongodb
getting No.2 item, name:PRIVATHINKER Sinicism Store Denim Men Harem Jeans Pants Male Drawstring Belt Vintage
getting item reviews:8
getting item price:$34.19 

getting item orders number:21
getting item url: https://www.aliexpress.com/item/New-Arrival-Men-Casual-American-USA-Flag-Printed-Jeans-Pants-Mens-Graffiti-Print-white-hip-hop/32801279593.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=4d313ff1-32bd-4bfa-93ee-f4594f55b838-21&algo_pvid=4d313ff1-32bd-4bfa-93ee-f4594f55b838 
save to mongodb
getting No.23 item, name:INFLATION 2018 Frayed For Men Skinny Destroyed stripe Ripped Slim Fit Jeans Pants
getting item reviews:3
getting item price:$34.21
getting item orders number:21
getting item url: https://www.aliexpress.com/item/INFLATION-2018-Ripped-Frayed-Pants-For-Men-Skinny-Destroyed-stripe-Ripped-Elastic-Slim-Fit-Jeans-Casual/32896909590.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10

save to mongodb
getting No.40 item, name:PARKLEES Casual Black Skinny Jeans Men 2018 Stretch Slim Fit Denim Style Jeans
getting item reviews:7
getting item price:$16.79 - 17.75
getting item orders number:20
getting item url: https://www.aliexpress.com/item/Casual-Black-Skinny-Jeans-Men-2018-Brand-Stretch-Slim-Fit-Denim-Trousers-Pantalon-Homme-Jean-Hip/32884993043.html?ws_ab_test=searchweb0_0searchweb201602_3_10065_10068_10890_319_10546_317_10548_10696_453_10084_454_10083_433_10618_431_10307_537_536_10059_10884_10887_100031_321_322_10103searchweb201603_53ppcSwitch_0&algo_expid=4d313ff1-32bd-4bfa-93ee-f4594f55b838-39&algo_pvid=4d313ff1-32bd-4bfa-93ee-f4594f55b838 
save to mongodb
getting No.41 item, name:neverfunction 2016 4 style men Spring Autumn camouflage American flag jeans pants
getting item reviews:8
getting item price:$16.45
getting item orders number:20
getting item url: https://www.aliexpress.com/item/2016-new-4-style-men-Spring-Autumn-camouflage-American-flag-printed-jeans-fas